In [1]:
from transformers import BertForSequenceClassification, BertTokenizer

import torch
import pandas as pd
from pipeline import create_dataloaders, train_bert, evaluate_bert
from torch.optim import Adam

import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/Users/chi/.venvs/env_torch/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = 'sampled_preprocessed.json'
df = pd.read_json(file_path)

In [3]:
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
bert_model = bert_model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 32
max_len = 128

train_loader, val_loader, test_loader = create_dataloaders(df, tokenizer, batch_size=batch_s
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           )


                                                           

optimizer = Adam(bert_model.parameters(), lr=1e-3)


In [5]:
train_losses = []
train_accuracies = []
val_accuracies = []

n_epochs = 5  # Set the number of epochs
for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    
    train_loss, train_acc = train_bert(bert_model, train_loader, optimizer, device)
    val_acc = evaluate_bert(bert_model, val_loader, device)
    
    # Append metrics
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    
    print(f'Epoch {epoch+1}: Train Loss {train_loss:.4f}, Train Acc {train_acc:.4f}, Val Acc {val_acc:.4f}')

Epoch 1/5


Evaluating: 100%|██████████| 9/9 [00:13<00:00,  1.47s/it, acc=0.737] 


Epoch 1: Train Loss 0.6692, Train Acc 0.6892, Val Acc 0.7374
Epoch 2/5


Evaluating: 100%|██████████| 9/9 [00:13<00:00,  1.50s/it, acc=0.737] 


Epoch 2: Train Loss 0.5939, Train Acc 0.7343, Val Acc 0.7374
Epoch 3/5


...

Epoch 3: Train Loss 0.5909, Train Acc 0.7356, Val Acc 0.7374
Epoch 4/5


Evaluating: 100%|██████████| 9/9 [00:13<00:00,  1.49s/it, acc=0.737] 


Epoch 4: Train Loss 0.6095, Train Acc 0.7278, Val Acc 0.7374
Epoch 5/5


Evaluating: 100%|██████████| 9/9 [00:13<00:00,  1.49s/it, acc=0.737] 

Epoch 5: Train Loss 0.5874, Train Acc 0.7362, Val Acc 0.7374
